In [1]:
%pylab nbagg

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import psycopg2
from matplotlib import animation
# import cartopy.crs as ccrs
from datetime import datetime, timedelta
from pathlib import Path
from os import listdir

In [3]:
from fastai import *
from fastai.tabular import *

ModuleNotFoundError: No module named 'numpy.testing.decorators'

In [5]:
orig_data = pd.read_csv('data_311.csv')

C:\Users\efreiling\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
orig_data.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,30344685,2015-04-06 09:12:00,2015-04-08 12:00:00,DSNY,BCC - Brooklyn North,Missed Collection (All Materials),2 Bulk-Missed Collection,Sidewalk,11236.0,15 EAST 88 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.650824,-73.920302,"(40.650823560883765, -73.9203020905484)"
1,18336946,2010-07-24 20:37:00,2010-07-24 20:50:00,DEP,Department of Environmental Protection,Water System,Fire Hydrant Emergency (FHE),NaN,11237.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.701832,-73.919321,"(40.7018317851356, -73.91932144034594)"
2,21077563,2011-08-21 02:15:14,2011-08-21 05:10:12,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11233.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.683498,-73.935154,"(40.68349849847838, -73.93515400522416)"
3,38448155,2018-02-12 12:43:30,2018-02-13 01:31:11,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11209.0,7201 RIDGE BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.634049,-74.029666,"(40.63404926192071, -74.02966617941425)"
4,21743239,2011-10-26 16:41:45,2011-11-01 10:24:46,DOT,Department of Transportation,Broken Muni Meter,No Receipt,Street,10023.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.780561,-73.980209,"(40.780561348235594, -73.98020912090448)"


In [8]:
from importlib import reload
import utils

ModuleNotFoundError: No module named 'numpy.testing.decorators'

In [32]:
cont_cols = [
    'x_coordinate_state_plane',
    'y_coordinate_state_plane',
    'latitude',
    'longitude',
    'time_to_close',
    'due_len',
    'time_over',
    'updated_time'
]
cat_cols = [
    'agency',
    'borough',
    'location_type',
    'incident_zip',
    'street_name',
    'cross_street_1',
    'cross_street_2',
    'intersection_street_1',
    'intersection_street_2',
    'address_type',
    'city',
    'landmark',
    'facility_type',
    'status',
    'community_board',
    'open_data_channel_type',
    'park_facility_name',
    'park_borough',
    'vehicle_type',
    'taxi_company_borough',
    'taxi_pick_up_location',
    'bridge_highway_name',
    'bridge_highway_direction',
    'road_ramp',
    'bridge_highway_segment',
]

dep_var = ['complaint_type']

In [36]:
created_cols = list(set(df.columns) - set(cont_cols) - set(cat_cols) - set(dep_var))
all_cat_cols = cat_cols + created_cols

In [44]:
cont_cols[0] in nan_cols

True

In [47]:
set(df.columns[df.isna().any()]) & set(cont_cols)

{'due_len',
 'latitude',
 'longitude',
 'time_over',
 'time_to_close',
 'x_coordinate_state_plane',
 'y_coordinate_state_plane'}

In [21]:
df['latitude'] = df['latitude'].astype('double')
df['longitude'] = df['longitude'].astype('double')

# Fastai 

In [19]:
train_perc = 0.8
valid_idx = int(len(df) * train_perc)
# train_df, valid_df = df.iloc[:valid_idx].copy(), df.iloc[valid_idx:].copy()
# len(train_df), len(valid_df)

In [20]:
procs = [FillMissing, Categorify, Normalize] # FillMissing
dep_var = 'complaint_type'

In [22]:
data = (TabularList.from_df(df, path='.', cat_names=cat_cols, cont_names=cont_cols, procs=procs)
                        .split_by_idx(list(range(valid_idx,len(df))))
                        .label_from_df(cols = dep_var)
                        .databunch())

In [23]:
data.show_batch(rows=10)

agency,location_type,incident_zip,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,community_board,open_data_channel_type,park_facility_name,park_borough,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,x_coordinate_state_plane_na,y_coordinate_state_plane_na,latitude_na,longitude_na,time_to_close_na,due_len_na,time_over_na,updated_time_na,x_coordinate_state_plane,y_coordinate_state_plane,latitude,longitude,time_to_close,due_len,time_over,updated_time,target
HPD,RESIDENTIAL BUILDING,11433.0,SOUTH ROAD,164 STREET,165 STREET,,,ADDRESS,JAMAICA,,N/A,Closed,0 Unspecified,UNKNOWN,Unspecified,Unspecified,,,,,,,,False,False,False,False,False,True,True,False,1.8138,-0.4283,-0.4298,1.8127,0.0434,0.0663,0.0239,-0.1553,GENERAL CONSTRUCTION
DEP,,10032.0,AMSTERDAM AVENUE,W 166 ST,W 167 ST,,,ADDRESS,NEW YORK,,N/A,Closed,12 MANHATTAN,UNKNOWN,Unspecified,MANHATTAN,,,,,,,,False,False,False,False,False,True,True,False,-0.1950,1.1900,1.1905,-0.1943,0.0415,0.0663,0.0239,-0.2035,Water System
HPD,RESIDENTIAL BUILDING,11206.0,BOERUM STREET,BUSHWICK PLACE,WHITE STREET,,,ADDRESS,BROOKLYN,,N/A,Closed,0 Unspecified,UNKNOWN,Unspecified,Unspecified,,,,,,,,False,False,False,False,False,True,True,False,-0.1875,-0.3586,-0.3581,-0.1886,0.0529,0.0663,0.0239,0.0887,ELECTRIC
DOT,,11363.0,,,,42 AVENUE,LITTLE NECK PARKWAY,INTERSECTION,LITTLE NECK,,N/A,Closed,11 QUEENS,UNKNOWN,Unspecified,QUEENS,,,,,,,,False,False,False,False,False,True,True,False,2.5599,0.3964,0.3936,2.5627,0.0414,0.0663,0.0239,-0.2142,Street Light Condition
DPR,Street,11213.0,MONTGOMERY STREET,KINGSTON AVENUE,ALBANY AVENUE,,,ADDRESS,BROOKLYN,,N/A,Closed,09 BROOKLYN,PHONE,Unspecified,BROOKLYN,,,,,,,,False,False,False,False,False,False,False,False,-0.2434,-0.8444,-0.8439,-0.2450,0.0483,0.0816,0.0341,-0.0376,Damaged Tree
HPD,RESIDENTIAL BUILDING,10463.0,JACOBUS PLACE,WEST 225 STREET,FT CHARLES PLACE,,,ADDRESS,NEW YORK,,N/A,Closed,0 Unspecified,UNKNOWN,Unspecified,Unspecified,,,,,,,,False,False,False,False,False,True,True,False,0.1643,1.6182,1.6184,0.1664,0.0434,0.0663,0.0239,-0.1553,HEATING
NYPD,Street/Sidewalk,11385.0,73 STREET,CENTRAL AVENUE,COOPER AVENUE,,,ADDRESS,RIDGEWOOD,,Precinct,Closed,05 QUEENS,PHONE,Unspecified,QUEENS,,,,,,,,False,False,False,False,False,False,False,False,0.6382,-0.3727,-0.3728,0.6372,0.0415,0.0639,0.0239,-0.2163,Blocked Driveway
HPD,RESIDENTIAL BUILDING,11365.0,56 AVENUE,175 STREET,175 PLACE,,,ADDRESS,FRESH MEADOWS,,N/A,Closed,0 Unspecified,UNKNOWN,Unspecified,Unspecified,,,,,,,,False,False,False,False,False,True,True,False,1.7931,0.0691,0.0676,1.7938,0.0620,0.0663,0.0239,0.3199,GENERAL CONSTRUCTION
HPD,RESIDENTIAL BUILDING,11212.0,EAST 98 STREET,NEWPORT STREET,SARATOGA AVENUE,,,ADDRESS,BROOKLYN,,N/A,Closed,0 Unspecified,UNKNOWN,Unspecified,Unspecified,,,,,,,,False,False,False,False,False,True,True,False,0.1153,-0.9215,-0.9212,0.1134,0.0434,0.0663,0.0239,-0.1553,ELECTRIC
DOT,,11221.0,DE KALB AVENUE,EVERGREEN AVENUE,MYRTLE AVENUE,,,ADDRESS,BROOKLYN,,N/A,Closed,Unspecified BROOKLYN,UNKNOWN,Unspecified,BROOKLYN,,,,,,,,False,False,False,False,False,True,True,False,-0.0390,-0.4693,-0.4689,-0.0401,0.0417,0.0663,0.0239,-0.1981,Street Condition


In [172]:
learn = tabular_learner(data, layers=[200, 100], metrics=accuracy)
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 